In [21]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
from keras.utils import to_categorical 
from sklearn.metrics import precision_recall_fscore_support

In [22]:
ba = ['single', 'double', 'home_run', 'field_error', 'triple', 'fielders_choice', 'run']

In [23]:
obp = ['single', 'double', 'home_run',
       'hit_by_pitch', 'field_error','walk', 'triple',
       'fielders_choice', 'intent_walk', 'run', 'catcher_interf']

In [24]:
df = pd.read_csv('pitcher_data/545333_data.csv')
df = df.drop(columns=df.columns[:5].append(df.columns[9:])).dropna()

obp_df = df[~df['events'].str.contains('caught|pickoff|defensive')==True]
obp_df['obp'] = obp_df.events.isin(obp).astype(int)

ba_df = df[~df['events'].str.contains('caught|pickoff|defensive|hit_by|field_error|walk|catcher_interf')==True]
ba_df['ba'] = ba_df.events.isin(ba).astype(int)

full_obp_df = obp_df
full_ba_df = obp_df

/Users/marko/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/marko/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
obp_df.head()

,player_name,batter,pitcher,events,obp
0,Trevor Bauer,643446,545333,field_out,0
3,Trevor Bauer,607043,545333,field_out,0
6,Trevor Bauer,476704,545333,strikeout,0
12,Trevor Bauer,642708,545333,single,1
14,Trevor Bauer,453943,545333,field_out,0


In [26]:
obp_df = obp_df.groupby(['batter', 'pitcher', 'obp']).size().reset_index(name='obp_counts')

def obp(row, df):
    if row.obp:
        if df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & (row.obp==0)].empty:
            return 1
        else:
            
return row.obp_counts/ (row.obp_counts + df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & row.obp==0]['obp_counts'])
    else:
        if df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & row.obp==1].empty:
            return 0
        else:
            return row.obp_counts/ (row.obp_counts + df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & row.obp==1]['obp_counts'])
obp_df['obp_rate'] = obp_df.apply(lambda row: obp(row, obp_df), axis=1)

In [29]:
def total_counts(row, df):
    if df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & (df.obp!=row.obp)].empty:
        return row.obp_counts
    else:
        return df[(df.batter == row.batter) & (df.pitcher == row.pitcher) & (df.obp!=row.obp)]['obp_counts'] + row.obp_counts
obp_df['total_counts'] = obp_df.apply(lambda row: total_counts(row, obp_df), axis=1)

In [30]:
obp_df

,batter,pitcher,obp,obp_counts,total_counts
0,110029,545333,0,3,3
1,116338,545333,0,11,[18]
2,116338,545333,1,7,[18]
3,116380,545333,0,2,[6]
4,116380,545333,1,4,[6]
5,116539,545333,0,5,[6]
6,116539,545333,1,1,[6]
7,116706,545333,1,2,2
8,120074,545333,0,3,[7]
9,120074,545333,1,4,[7]


In [5]:
for filename in os.listdir('pitcher_data/'):
    if filename.endswith('.csv') and filename != '545333_data.csv':
        df = pd.read_csv('pitcher_data/'+filename)
        df = df.drop(columns=df.columns[:5].append(df.columns[9:])).dropna()
        #df = df[~df['events'].str.contains('caught|pickoff|defensive')==True]
        #df['obp'] = df.events.isin(obp).astype(int)
        df = df[~df['events'].str.contains('caught|pickoff|defensive|hit_by|field_error|walk|catcher_interf')==True]
        #df = df[df['game_date'].str.contains('2019')==True]
        df['ba'] = df.events.isin(ba).astype(int)
        full_df = full_df.append(df)

/Users/marko/anaconda3/envs/news/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,43,78,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
pitchers = pd.read_csv('2019_pitchers.csv')
batters = pd.read_csv('2019_batters.csv')

In [67]:
#obp = pd.read_csv('obp_non_aggregate.csv')

In [23]:
a = full_df.drop(columns=['player_name', 'events'])
a['batter'] = a.batter.astype(int)
b = a
b = a.groupby(['batter', 'pitcher'], as_index=False).filter(lambda x: len(x)>10)

In [24]:
c = b.groupby(['batter', 'pitcher'], as_index=False)[["ba"]].mean()
new_df = c.reset_index()
final_df = new_df[new_df.batter.isin(batters.player_id)]

In [25]:
final_df = final_df.drop(columns='index')

In [26]:
final_df['batter_ba'] = final_df.batter.apply(lambda x: batters[batters['player_id']==x]['ba'].values[0])
final_df['pitcher_ba'] = final_df.pitcher.apply(lambda x: pitchers[pitchers['player_id']==x]['ba'].values[0])

In [37]:
final_df.head()

,batter,pitcher,ba,batter_ba,pitcher_ba,ba_cat,pitcher_factorized,batter_factorized,ba_factorized
1415,400085,282332,0.200000,0.0,0.265,middle,0,0,0
1416,400085,407845,0.388889,0.0,0.269,high,1,0,1
1417,400085,424144,0.400000,0.0,0.242,high,2,0,1
1418,400085,425844,0.272727,0.0,0.228,high,3,0,1
1419,400085,429719,0.181818,0.0,0.351,middle,4,0,0


In [28]:
def my_func(ba, batter_ba, pitcher_ba):
    if (ba > batter_ba) & (ba > pitcher_ba):
        return 'high'
    elif (ba < batter_ba) & (ba < pitcher_ba):
        return 'low'
    else:
        return 'middle'

In [29]:
final_df['ba_cat'] = final_df.apply(lambda row: my_func(row['ba'], row['batter_ba'], row['pitcher_ba']), axis=1)

In [30]:
final_df.ba_cat.value_counts()

high      5584
low       5110
middle    2046
Name: ba_cat, dtype: int64

In [31]:
final_df['pitcher_factorized'] = pd.factorize(final_df['pitcher'])[0]
final_df['batter_factorized'] = pd.factorize(final_df['batter'])[0]
final_df['ba_factorized'] = pd.factorize(final_df['ba_cat'])[0]
num_batters = len(pd.unique(final_df.batter))
num_pitchers = len(pd.unique(final_df.pitcher))

# Model

In [54]:
def build_model(): 
    input_pitcher = layers.Input(shape=(1,))
    pitcher_embedding = layers.Embedding(num_pitchers, 5)(input_pitcher)
    input_batter = layers.Input(shape=(1,))
    batter_embedding = layers.Embedding(num_batters, 5)(input_batter)
    pitcher_flat = layers.Flatten()(pitcher_embedding)
    batter_flat = layers.Flatten()(batter_embedding)
    cl_concat = layers.Concatenate()([pitcher_flat, batter_flat])
    cl_dense = layers.Dense(100, activation='relu')(cl_concat)
    cl_dense1 = layers.Dense(20, activation='relu')(cl_dense)
    pred = layers.Dense(3, activation="softmax")(cl_dense1)
    model = Model([input_pitcher, input_batter], pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model

In [55]:
train_pitcher, test_pitcher, train_batter, test_batter, train_labels, test_labels = train_test_split(final_df['pitcher_factorized'], final_df['batter_factorized'], to_categorical(final_df['ba_factorized']), test_size=0.1)

In [62]:
model = None
model = build_model()
history = model.fit([train_pitcher, train_batter],
          train_labels,
         validation_data=([test_pitcher, test_batter], test_labels),
          epochs=10,
          batch_size=128,
          class_weight={0:2, 1:1, 2:1}
         )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 1, 5)         1930        input_23[0][0]                   
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 1, 5)         2160        input_24[0][0]                   
__________________________________________________________________________________________________
flatten_23

In [60]:
preds = model.predict([test_pitcher, test_batter])

In [61]:
print(np.argmax(preds, axis=1)[:20])
print(np.argmax(test_labels,axis=1)[:20])
precision_recall_fscore_support(np.argmax(test_labels,axis=1), np.argmax(preds, axis=1), average='macro')

[2 0 0 2 0 1 2 1 2 2 2 2 1 1 1 1 2 2 1 1]
[1 0 2 1 2 1 1 1 1 2 2 2 2 1 2 0 2 1 2 2]


(0.3823499508028616, 0.3929115949377804, 0.3819894031409005, None)

# Extra